In [11]:
library(renv)
# renv::restore()
library(GGally)
library(tidymodels)
library(tidyverse)
library(leaps)
library(caret)
library(kknn)
library(randomForest)

In [53]:
articles <- read_csv('data/OnlineNewsPopularity.csv')
head(data)

Rows: 39644 Columns: 61
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): url
dbl (60): timedelta, n_tokens_title, n_tokens_content, n_unique_tokens, n_no...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


                                                                            
1 function (..., list = character(), package = NULL, lib.loc = NULL,        
2     verbose = getOption("verbose"), envir = .GlobalEnv, overwrite = TRUE) 
3 {                                                                         
4     fileExt <- function(x) {                                              
5         db <- grepl("\\\\.[^.]+\\\\.(gz|bz2|xz)$", x)                     
6         ans <- sub(".*\\\\.", "", x)                                      

In [54]:
articles_temp <- articles

drop_columns <- c('url', 'weekday_is_sunday', 'is_weekend')

num_columns <- c('n_tokens_title', 'n_tokens_content', 'num_hrefs','num_self_hrefs',
                 'num_imgs','num_videos','average_token_length','num_keywords','kw_min_min','kw_max_min','kw_avg_min','kw_min_max','kw_max_max','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess')

one_hot_columns <- c('data_channel_is_lifestyle','data_channel_is_entertainment','data_channel_is_bus','data_channel_is_socmed','data_channel_is_tech','data_channel_is_world','weekday_is_monday','weekday_is_tuesday','weekday_is_wednesday','weekday_is_thursday','weekday_is_friday','weekday_is_saturday')

passthrough_columns <- c('n_unique_tokens','n_non_stop_words','n_non_stop_unique_tokens','LDA_00','LDA_01','LDA_02','LDA_03','LDA_04','global_subjectivity','global_sentiment_polarity','global_rate_positive_words','global_rate_negative_words','rate_positive_words','rate_negative_words','avg_positive_polarity','min_positive_polarity','max_positive_polarity','avg_negative_polarity','min_negative_polarity','max_negative_polarity','title_subjectivity','title_sentiment_polarity','abs_title_subjectivity','abs_title_sentiment_polarity')

# One hot encoding
# articles_temp[one_hot_columns] <- lapply(articles[one_hot_columns], factor)
# a <- paste(" ~ ",paste(one_hot_columns, collapse="+"),sep = "")
# dmy <- dummyVars(a, data = articles)
# trsf <- data.frame(predict(dmy, newdata = articles))

#Log scaling
for(value in num_columns){
    articles_temp[value][articles_temp[value] == 0] = 1
}

articles_temp[num_columns] <- log(articles_temp[num_columns])

# articles <- articles[, !(names(articles) %in% drop_columns)]

Warning message in FUN(X[[i]], ...):
"NaNs produced"
Warning message in FUN(X[[i]], ...):
"NaNs produced"
Warning message in FUN(X[[i]], ...):
"NaNs produced"


In [55]:
articles_temp[num_columns]

n_tokens_title,n_tokens_content,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2.484907,5.389072,1.3862944,0.6931472,0.000000,0.000000,1.543376,1.609438,0,0,0,0,0,0,0,0,0,6.206576,6.206576,6.206576
2.197225,5.541264,1.0986123,0.0000000,0.000000,0.000000,1.592032,1.386294,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000
2.197225,5.351858,1.0986123,0.0000000,0.000000,0.000000,1.480095,1.791759,0,0,0,0,0,0,0,0,0,6.822197,6.822197,6.822197
2.197225,6.274762,2.1972246,0.0000000,0.000000,0.000000,1.482717,1.945910,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000
2.564949,6.977281,2.9444390,2.9444390,2.995732,0.000000,1.543904,1.945910,0,0,0,0,0,0,0,0,0,6.300786,9.680344,8.055525
2.302585,5.913503,0.6931472,0.6931472,0.000000,0.000000,1.472348,2.197225,0,0,0,0,0,0,0,0,0,9.047821,9.047821,9.047821
2.079442,6.866933,3.0445224,2.9957323,2.995732,0.000000,1.537763,2.302585,0,0,0,0,0,0,0,0,0,6.300786,9.680344,8.055525
2.484907,6.896694,2.9957323,2.9957323,2.995732,0.000000,1.529917,2.197225,0,0,0,0,0,0,0,0,0,6.300786,9.680344,8.055525
2.397895,4.574711,0.6931472,0.0000000,0.000000,0.000000,1.580147,1.945910,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000


In [33]:
articles_temp


url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,⋯,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
http://mashable.com/2013/01/07/amazon-instant-video-browser/,731,2.484907,5.389072,0.6635945,1,0.8153846,1.3862944,0.6931472,0.000000,⋯,0.10000000,0.7000000,-0.3500000,-0.6000000,-0.2000000,0.5000000,-0.1875000,0.00000000,0.1875000,593
http://mashable.com/2013/01/07/ap-samsung-sponsored-tweets/,731,2.197225,5.541264,0.6047431,1,0.7919463,1.0986123,0.0000000,0.000000,⋯,0.03333333,0.7000000,-0.1187500,-0.1250000,-0.1000000,0.0000000,0.0000000,0.50000000,0.0000000,711
http://mashable.com/2013/01/07/apple-40-billion-app-downloads/,731,2.197225,5.351858,0.5751295,1,0.6638655,1.0986123,0.0000000,0.000000,⋯,0.10000000,1.0000000,-0.4666667,-0.8000000,-0.1333333,0.0000000,0.0000000,0.50000000,0.0000000,1500
http://mashable.com/2013/01/07/astronaut-notre-dame-bcs/,731,2.197225,6.274762,0.5037879,1,0.6656347,2.1972246,0.0000000,0.000000,⋯,0.13636364,0.8000000,-0.3696970,-0.6000000,-0.1666667,0.0000000,0.0000000,0.50000000,0.0000000,1200
http://mashable.com/2013/01/07/att-u-verse-apps/,731,2.564949,6.977281,0.4156456,1,0.5408895,2.9444390,2.9444390,2.995732,⋯,0.03333333,1.0000000,-0.2201923,-0.5000000,-0.0500000,0.4545455,0.1363636,0.04545455,0.1363636,505
http://mashable.com/2013/01/07/beewi-smart-toys/,731,2.302585,5.913503,0.5598886,1,0.6981982,0.6931472,0.6931472,0.000000,⋯,0.13636364,0.6000000,-0.1950000,-0.4000000,-0.1000000,0.6428571,0.2142857,0.14285714,0.2142857,855
http://mashable.com/2013/01/07/bodymedia-armbandgets-update/,731,2.079442,6.866933,0.4181626,1,0.5498339,3.0445224,2.9957323,2.995732,⋯,0.10000000,1.0000000,-0.2244792,-0.5000000,-0.0500000,0.0000000,0.0000000,0.50000000,0.0000000,556
http://mashable.com/2013/01/07/canon-poweshot-n/,731,2.484907,6.896694,0.4335736,1,0.5721078,2.9957323,2.9957323,2.995732,⋯,0.10000000,1.0000000,-0.2427778,-0.5000000,-0.0500000,1.0000000,0.5000000,0.50000000,0.5000000,891
http://mashable.com/2013/01/07/car-of-the-future-infographic/,731,2.397895,4.574711,0.6701031,1,0.8367347,0.6931472,0.0000000,0.000000,⋯,0.40000000,0.8000000,-0.1250000,-0.1250000,-0.1250000,0.1250000,0.0000000,0.37500000,0.0000000,3600


In [27]:
articles['num_self_hrefs'][articles['num_self_hrefs'] == 0] = 1
articles$num_self_hrefs

[1]   2   1   1   1  19   2  20  20   1   1   1   1   2   2   3   1  18   1
   [19]   1  24  19   2  20   1   2   2   3  22   1   1   3   2   1   1   2   4
   [37]  22   7   6   4  21   1  24  19  20   1  24   4  21  20   1  21  21   3
   [55]  21   6   1   4   8   5  19   1  20   1  21   3   1   6   3   2   1   4
   [73]   1   3  19   1   3  10  11   2   1   2   4   1   2  20   3   2   2  20
   [91]  20   1   1   9   1   5   4  20   1   2   2  21  19  30   1   1   1   3
  [109]   4  23   3   1   1   3  24   1   6   2   5   1   4   4   1   5   9   2
  [127]   4   2   1   1   1  20  19   1  22  10   1   3   1   2   1  10   1   1
  [145]   3   1   1   4   8   2   6   1   2   2   3   2   2  22   1   1   2   2
  [163]  21  20   1   2   7  21   1   1   1  21   1   2   3   8   1  23   6   5
  [181]   6   4   1   3   3   2   1   1   2   5   3   9  20  38   7   4   8   1
  [199]   2   7   5   1   5   1   2   4   2  27   1   2   3   3   7  12   1   1
  [217]   5   1   3   2   1   1   7   7  27   1  21   1   1   7   2   2   4   2
  [235]   1   1   1   4   3   1   3   1   2   2   1   1  23  19   3   1   1   2
  [253]   3   2   3   2   1   2   3   1   8   1   1   1   3   2   4   7  39   3
  [271]   1   1   2   4   6   3   7   2   1   1   6   8   2   1   1   2   1   1
  [289]   1   5  23  22   4   1   1   2   1  20   3  23   1   5   1   1   2   3
  [307]  15   1   4   3   3   1   6   2   1   2  47   5  12   1   1  21   1  19
  [325]  11  20   4   3   1  31  20   1   5   3   1   1  24   2   3   5   3   1
  [343]   3   3   3   2   1   5   2   1  19   1   3   1   1   1   1   3   1   2
  [361]   4   1   5   1   1  11   1   1   3   4   1   1   1   2   2   1   1   4
  [379]   1   1   1   1   1   2   3   4   5   3   3   6   2   5   2   7  24   8
  [397]   4   5   1   2   2   6   5   2   3   3   1   2   1   2   3   5   2   2
  [415]   2   6   5   5   3   3   1   7   7   4   2   1  28   3   3   1   1   2
  [433]   2   4   8   1   6   7   7   2   8   1   4   1   1   2   2   1   3   1
  [451]   1   3   1   3   3   2   5   4   2   1   1   3  11   2   2   1   3   3
  [469]   3   1   2   1   2   1   6   1   1   2   3   1   2   1   1   2   2   4
  [487]   1   3   3   1   6   7   2   4  28   4   1   2   3   2   3   3   4   1
  [505]   2   1   9   2   1   4   1   2   4   1   1   4   3  12   1   3   1   1
  [523]   4   1   1   3   8   2   2   4   1   1   4   6   6   3   4   4   3   2
  [541]   4   1   1   4   1   1   1   1   1   1   1   1   1   1   2   4   1   3
  [559]   2   2   1   1   4   1   1   5   2   1   3   4   3   3   1   1   4   2
  [577]   5   2   1   5   3   3   6   4   3   1   2   7   5   1   3   3   3   1
  [595]   1   1   3   1   7   2   3   4   2   1   3   1   9   1   1   4   3   2
  [613]   2   4   1   3   4   1   3   4  16   1   5   1   1   1   1   4   8   1
  [631]   1   1   4   1   6   3  12   2   1   2   1   5   5   3   4   1   3   3
  [649]   1   1   8   2   1   2   2   4   1   5   1   3   5   7   2   2   1   1
  [667]   1   3   1   5   5   3   5   1   1   1   7   4   1   3   3   4   3   5
  [685]   3   1   1   3   7   5   6   2   2   1   2   1   1  18   2   4   1   4
  [703]   1   5   1   6   4   1   1   1  41   2   4   2   2   9   2   3   1   1
  [721]   3   1   3   1   1   2   1   1   1   1   2   2   3   4   1   3   1   1
  [739]   1   5   1   2   3   2   3   3   1   1   1   2   1   2   5   3   4   1
  [757]  12   3   3   3   6   5   6   4   4   1   6   2   4   4   1   7   3   1
  [775]   2   1   1   1   3   7   1   1   3   1   5   5   3   1   2   3   2   7
  [793]   4   1   2   5   4   5  12   1   1   1   1   2  34   2   3   2   1   4
  [811]   2   2   7   3   1   2   1   1   1   1   1   3   4   1   1   4   3   6
  [829]   1   2   1   2   1   5   3   1   5   4   1   1   1   4   4   3   3   3
  [847]   1   1   4   3   3   2   3   7   3   5   1   1   3   1   3   1   4   3
  [865]   1   3   1   1   1   2   9   3   8   1   1   1   1   7   3   1   2   2
  [883]   1   1   2   3   1   4   5   1   2   3   6   1   6   2   2   5   2   1
  [9

In [21]:
for(value in num_columns){
    
}

[1] n_tokens_title
[1] n_tokens_content
[1] num_hrefs
[1] num_self_hrefs
[1] num_imgs
[1] num_videos
[1] average_token_length
[1] num_keywords
[1] kw_min_min
[1] kw_max_min
[1] kw_avg_min
[1] kw_min_max
[1] kw_max_max
[1] kw_avg_max
[1] kw_min_avg
[1] kw_max_avg
[1] kw_avg_avg
[1] self_reference_min_shares
[1] self_reference_max_shares
[1] self_reference_avg_sharess


In [6]:
set.seed(2024)

articles$ID <- 1:nrow(articles)

training_articles <- sample_n(articles, size = nrow(articles) * 0.70, replace = FALSE)

testing_articles <- anti_join(articles, training_articles, by = "ID")

head(training_articles, 3)
nrow(training_articles)

head(testing_articles, 3)
nrow(testing_articles)

timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,⋯,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,ID
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
317,9,549,0.4935305,1,0.6827586,8,7,2,0,⋯,1,-0.2941358,-0.8,-0.125,0.5,0.5,0.0,0.5,1500,21029
338,9,452,0.5193622,1,0.6000000,18,1,13,0,⋯,1,-0.2171875,-0.4,-0.050,0.0,0.3,0.5,0.3,1400,19872
584,7,1361,0.3924890,1,0.6248237,8,3,1,1,⋯,1,-0.2574653,-1.0,-0.050,0.5,0.5,0.0,0.5,1500,7802


[1] 27750

timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,⋯,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,ID
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
731,9,255,0.6047431,1,0.7919463,3,1,1,0,⋯,0.7,-0.1187500,-0.125,-0.1000000,0,0,0.5,0,711,2
731,9,211,0.5751295,1,0.6638655,3,1,1,0,⋯,1.0,-0.4666667,-0.800,-0.1333333,0,0,0.5,0,1500,3
731,9,285,0.7441860,1,0.8415301,4,2,0,21,⋯,1.0,-0.3378889,-0.700,-0.1000000,1,-1,0.5,1,10000,14


[1] 11894

Going to fit an initial linear model using all attributes and ordinary least squares(OLS) to receive a baseline model

In [7]:
article_full_OLS <- lm(shares ~ ., data = training_articles[,-61])

tidy(article_full_OLS)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-1.038884e+05,8.469753e+06,-0.01226582,9.902136e-01
timedelta,1.246653e+00,4.601193e-01,2.70941303,6.744383e-03
n_tokens_title,1.120513e+02,3.397562e+01,3.29799151,9.750176e-04
n_tokens_content,5.355600e-01,2.617269e-01,2.04625534,4.074068e-02
n_unique_tokens,3.953962e+03,2.248320e+03,1.75862970,7.865147e-02
n_non_stop_words,-8.376189e+02,8.146429e+03,-0.10282038,9.181063e-01
n_non_stop_unique_tokens,-2.754471e+03,1.907788e+03,-1.44380339,1.488056e-01
num_hrefs,3.291022e+01,7.781449e+00,4.22931743,2.351536e-05
num_self_hrefs,-6.637569e+01,2.156097e+01,-3.07851133,2.082410e-03


In [8]:
articles_test_pred_full_OLS <- predict(article_full_OLS, newdata = testing_articles[, -61])
head(articles_test_pred_full_OLS)

1         2         3         4         5         6 
-440.1909  436.9545 1808.3143  511.9230  577.6396 -271.0715

In [9]:
articles_RMSE_models <- tibble(
    Model = 'OLS Full Regression',
    RMSE = RMSE(
        articles_test_pred_full_OLS,
        testing_articles$shares
    )
)
articles_RMSE_models

Model,RMSE
<chr>,<dbl>
OLS Full Regression,12032.83


In [18]:
confusionMatrix(factor(articles_test_pred_full_OLS), factor(testing_articles$shares))

ERROR: Error in confusionMatrix.default(factor(articles_test_pred_full_OLS), : the data cannot have more levels than the reference


Since we have multiple attributes, we wish to find the most significant attributes that contribute the most to the amount of shares an article receives. We will do so using forward selection by way of the `tidymodels` package.

In [10]:
articles_forward_sel <- regsubsets(
    x = shares ~ ., nvmax = 60,
    data = training_articles[, -61],
    method = 'forward' 
)

articles_forward_summary <- summary(articles_forward_sel)
articles_forward_summary

Warning message in leaps.setup(x, y, wt = wt, nbest = nbest, nvmax = nvmax, force.in = force.in, :
"2  linear dependencies found"


Reordering variables and trying again:


Warning message in rval$lopt[] <- rval$vorder[rval$lopt]:
"number of items to replace is not a multiple of replacement length"


Subset selection object
Call: regsubsets.formula(x = shares ~ ., nvmax = 60, data = training_articles[, 
    -61], method = "forward")
59 Variables  (and intercept)
                              Forced in Forced out
timedelta                         FALSE      FALSE
n_tokens_title                    FALSE      FALSE
n_tokens_content                  FALSE      FALSE
n_unique_tokens                   FALSE      FALSE
n_non_stop_words                  FALSE      FALSE
n_non_stop_unique_tokens          FALSE      FALSE
num_hrefs                         FALSE      FALSE
num_self_hrefs                    FALSE      FALSE
num_imgs                          FALSE      FALSE
num_videos                        FALSE      FALSE
average_token_length              FALSE      FALSE
num_keywords                      FALSE      FALSE
data_channel_is_lifestyle         FALSE      FALSE
data_channel_is_entertainment     FALSE      FALSE
data_channel_is_bus               FALSE      FALSE
data_channel_is_soc

In [12]:
articles_forward_summary_df <- tibble(
    n_input_variables = 1:57,
    RSQ = articles_forward_summary$rsq,
    RSS = articles_forward_summary$rss,
    ADJ.R2 = articles_forward_summary$adjr2,
    Cp = articles_forward_summary$cp,
    BIC = articles_forward_summary$bic,
)
articles_forward_summary_df

n_input_variables,RSQ,RSS,ADJ.R2,Cp,BIC
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.01200777,3.559806e+12,0.01197217,260.61524,-314.77045
2,0.01430799,3.551518e+12,0.01423694,197.41088,-369.22169
3,0.01547587,3.547310e+12,0.01536942,166.30496,-391.88933
4,0.01701876,3.541751e+12,0.01687705,124.56872,-425.18060
5,0.01810126,3.537851e+12,0.01792430,95.88316,-445.52584
6,0.01878867,3.535374e+12,0.01857646,78.39723,-454.72886
7,0.01936525,3.533297e+12,0.01911782,64.05269,-460.80936
8,0.01983188,3.531615e+12,0.01954922,52.82518,-463.78616
9,0.02024288,3.530134e+12,0.01992501,43.17466,-465.19352


Since we are doing predictive modelling, we will use Mallow's $C_p$ to assess the performance of the different predictive models, where the smaller the $C_p$ value is, the more precise the model relatively is. In this case, using 15 input variables has the smallest $C_p$ value.

The 15 input variables are:
```
n_tokens_title
n_tokens_content
num_hrefs
num_self_hrefs
average_token_length
data_channel_is_lifestyle
data_channel_is_entertainment
data_channel_is_bus
data_channel_is_socmed
data_channel_is_tech
data_channel_is_world
self_reference_min_shares
self_reference_max_shares
self_reference_avg_shares
weekday_is_wednesday

```

Now a reduced model is made using these 15 input variables.

In [13]:
articles_red_OLS <- lm(shares ~ n_tokens_title + n_tokens_content + num_hrefs + num_self_hrefs +
        average_token_length + data_channel_is_lifestyle + data_channel_is_entertainment + data_channel_is_bus + 
        data_channel_is_socmed +  data_channel_is_tech + data_channel_is_world + self_reference_min_shares + 
        self_reference_max_shares + self_reference_avg_sharess + weekday_is_wednesday,
        data = training_articles
    )

articles_test_pred_red_OLS <- predict(articles_red_OLS, newdata = testing_articles[, -61])

In [14]:
articles_RMSE_models <- rbind(
    articles_RMSE_models,
    tibble(
        Model = "OLS Reduced Regression",
        RMSE = RMSE(
            articles_test_pred_red_OLS,
            testing_articles$shares
        )
    )
)
articles_RMSE_models

Model,RMSE
<chr>,<dbl>
OLS Full Regression,12032.83
OLS Reduced Regression,12081.84


Will also use a K-nearest neighbours model to predict shares using `tidymodels` and 5-fold cross validation to choose the apporiate amount of neighbours using minimum RMSPE.

In [15]:
articles_recipe <- recipe(shares ~ n_tokens_title + n_tokens_content + num_hrefs + num_self_hrefs +
        average_token_length + data_channel_is_lifestyle + data_channel_is_entertainment + data_channel_is_bus + 
        data_channel_is_socmed +  data_channel_is_tech + data_channel_is_world + self_reference_min_shares + 
        self_reference_max_shares + self_reference_avg_sharess + weekday_is_wednesday, data = training_articles) %>%
        step_scale(all_predictors()) %>%
        step_center(all_predictors())

articles_spec <- nearest_neighbor(weight_func = 'rectangular',
                                  neighbors = tune()) %>%
    set_engine('kknn') %>%
    set_mode('regression')

articles_vfold <- vfold_cv(training_articles, v = 5, strata = shares)



In [16]:
gridvals <- tibble(neighbors = seq(1, 30))

articles_multi <- workflow() |>
  add_recipe(articles_recipe) |>
  add_model(articles_spec) |>
  tune_grid(articles_vfold, grid = gridvals) |>
  collect_metrics() |>
  filter(.metric == "rmse") |>
  filter(mean == min(mean))

articles_k <- articles_multi |>
              pull(neighbors)
articles_multi

neighbors,.metric,.estimator,mean,n,std_err,.config
<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
30,rmse,standard,11257.94,5,929.8378,Preprocessor1_Model30


We can compare the mean RMSE from K-Nearest Neighbours to the OLS models from earlier, and see that the K-Nearest Neighbours had a smaller RMSE, meaning it performs better in this scenario.